In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import SimpleITK as sitk
import mahotas as mt
import imageio

from PIL import Image
from matplotlib.patches import Rectangle
from scipy import linalg
from skimage.util.shape import view_as_windows
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from itertools import product
from skimage.feature.texture import greycomatrix
from collageradiomics import Collage, HaralickFeature, show_colored_image, InformationMeasureOfCorrelation1Interpretation, scale_array_for_image
from datetime import datetime

In [ ]:
# Read images
image_paths = ['../../sample_data/BrainSliceTumor.png', '../../sample_data/ImageSlice.png', '../../sample_data/ImageSlice2.png']
mask_paths = ['../../sample_data/BrainSliceTumorMask.png', '../../sample_data/ImageNonRectangularMask.png', '../../sample_data/ImageNonRectangularMask2.png']
images_list = []
masks_list = []
for path in image_paths:
    image_sitk = sitk.ReadImage(path)
    image_array = sitk.GetArrayFromImage(image_sitk)
    images_list.append(image_array)

for path in mask_paths:
    mask_sitk = sitk.ReadImage(path)
    mask_array = sitk.GetArrayFromImage(mask_sitk)
    masks_list.append(mask_array)

In [ ]:
collage = Collage.from_multiple_images(
    images_list, 
    masks_list, 
    svd_radius=5, 
    verbose_logging=True, 
    haralick_feature_list=[HaralickFeature.All], 
    log_sample_rate=500, 
    cooccurence_angles=[0, 1*np.pi/4, 2*np.pi/4, 3*np.pi/4, 4*np.pi/4, 5*np.pi/4, 6*np.pi/4, 7*np.pi/4],
    information_measure_correlation1_interpretation = InformationMeasureOfCorrelation1Interpretation.XMinusYVariance,
    haralick_window_size=-1,
    greylevels=64
)

In [ ]:
# Run CoLlage Algorithm.
full_images = collage.execute()

In [ ]:
import logging
logger = logging.getLogger('imageio').setLevel(logging.ERROR)

def scale_array_for_image(array_to_scale):
    flat_array = array_to_scale.flatten()
    minimum = float(min(flat_array))
    maximum = float(max(flat_array))
    array_range = maximum - minimum
    array_to_scale = array_to_scale - minimum
    array_to_scale /= array_range
    array_to_scale *= 255
    return array_to_scale

def write_haralick_image(single_collage, feature, directory, image_path):
    haralick = scale_array_for_image(single_collage.haralick_features[:,:,feature])
    haralick = haralick.astype('uint8')
    imageio.imwrite(f'{directory}/haralick-feature-{feature + 1}.png', plt.cm.jet(haralick))
    img = Image.open(f'{directory}/haralick-feature-{feature + 1}.png')
    img = img.convert('RGBA')
    pixdata = img.load()

    width, height = img.size
    for y in range(height):
        for x in range(width):
            if pixdata[x, y] == (0, 0, 127, 255) or pixdata[x, y] == (127, 0, 0, 255):
                pixdata[x, y] = (255, 255, 255, 0)

    img.save(f'{directory}/haralick-feature-{feature + 1}.png', 'PNG')
    background = Image.open(image_path)
    foreground = Image.open(f'{directory}/haralick-feature-{feature + 1}.png')

    background = background.convert('RGBA')
    background.paste(foreground, (single_collage.mask_min_x, single_collage.mask_min_y), foreground)
    background.show()
    background.save(f'{directory}/haralick-feature-{feature + 1}.png')

In [ ]:
import pathlib

now = datetime.now()
date_string = now.strftime("%m%d%Y-%H%M%S")

# Writes images to output_data directory in the root of the repository and show them.
for x in range(len(collage.collages)):
    single_collage = collage.collages[x]
    print(f'Original Image {x + 1}')
    display(Image.open(image_paths[x]))

    output_directory = f'../../output_data/{date_string}-image{x + 1}'

    pathlib.Path(output_directory).mkdir(parents=True, exist_ok=True) 

    for feature in range(13):
        write_haralick_image(single_collage, feature, output_directory, image_paths[x])
        print(f'Image {x + 1} Haralick Feature {feature + 1}')
        display(Image.open(f'{output_directory}/haralick-feature-{feature + 1}.png'))